In [20]:
import pandas as pd
import numpy as np
from config import *
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [23]:
df = pd.read_parquet(DATA_DIR / "anti_saccade_processed.pq")

In [69]:
reaction_time_df = (df
 .query("stimulus_active == True")
 .sort_values(by=["participant_id", "trial_id", "stand_time"])
 .assign(stimulus_time = lambda x: np.select([x.event == "FIXPOINT", x.event != "FIXPOINT"], [x.stand_time, None]))
 .ffill()
 .query("event == 'ESACC'")
 .groupby(["experiment","participant_id", "trial_id"])
 .first()
 .reset_index()
 .assign(reaction_time = lambda x: x.stand_start_time - x.stimulus_time)
 .groupby(["experiment","participant_id"])
 .agg(mean_reaction_time = ('reaction_time', 'mean'))
 .reset_index()
)

/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/971070164.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .ffill()


In [ ]:
features = (df.groupby(["experiment", "participant_id"])
 

)
 = (df_grouped[["peak_velocity", "amplitude", "duration", "avg_pupil_size"]]
 .agg([np.min, np.max, np.mean, np.median, np.std])
 .reset_index())

features = features.join(reaction_time_df, how='left', on = ["experiment", "participant_id"])


/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/1233278412.py:3: FutureWarning: The provided callable <function min at 0x1107d3240> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  .agg([np.min, np.max, np.mean, np.median, np.std])
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/1233278412.py:3: FutureWarning: The provided callable <function max at 0x1107d3100> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  .agg([np.min, np.max, np.mean, np.median, np.std])
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/1233278412.py:3: FutureWarning: The provided callable <function mean at 0x1107d3b00> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used d

MergeError: Not allowed to merge between different levels. (2 levels on the left, 1 on the right)

# Load outcome

In [14]:
demographics = pd.read_excel(DATA_DIR / "demographic_info.xlsx")[["ID", "Group"]]

demographics["y"] = (demographics["Group"] == "PATIENT").astype(int)

# Model training

In [ ]:
data = features.join(demographics, how = 'left', on='participant_id').drop("participant_id")
X_data = data.drop("y")
y_data = data["y"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=.2)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", XGBClassifier(n_estimators=10, max_depth=6, learning_rate=1, objective='binary:logistic'))
])

pipe.fit(X_train, y_train).score(X_test, y_test)

results=pd.DataFrame()
results['columns']=X_train.columns
results['importances'] = pipe["clf"].feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)

results